# Dados do Projeto

Inicialmente faz-se necessária a criação de pastas aonde serão armazenados os dados do projeto. Para tanto aplica-se o código abaixo, de um função anteriormente criada chamada _create_folders_, localizada no diretório que está no meu computador.

In [ ]:
%run '../codes/files/create_folders.py'
create_folders('', ['data', 'docs', 'maps'])

Após isso é necessário fazer o *download* os dados espaciais do projeto que, no caso concreto, trata-se da divisão municipal. Aqui optou-se por baixar os dados de duas fontes diferentes, sendo uma do **Instituto Brasileiro de Geografia e Estatistica (IBGE)** e a outra fonte é do **Instituto Geográfico e Cartográfico de São Paulo (IGC-SP)**.

Os dados do **IBGE** estão em escala 1:250.000 e atendem o objetivo do projeto, ou seja, que é demostrar quais municípios são geridos por quais órgãos/instituiçoes. Portanto, faz-se o *download* do arquivo zipado do **IBGE** e se descompacta o arquivo _shapefile_. Aproveitou-se para renomear o arquivo um nome mais inteligível.

In [ ]:
# Carregar pacotes
import os
import urllib.request
import shutil

# Define o nome do arquivo zip a ser salvo
file = 'SP_IBGE_250k'

# Define o nome do arquivo, com diretório, que será salvo
filename = os.path.join('data', file + '.zip')

# URL para baixar o arquivo zip
url = ('http://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/'+
       'malhas_municipais/municipio_2018/UFs/'+
       'SP/sp_municipios.zip')

# Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
       shutil.copyfileobj(response, out_file)

# Tira da pasta zipada e renomeia arquivos
%run '../codes/files/zip_file.py'
unzip_renaming(filename, file)

Processo similar foi feito com a divisão municipal disponibilizada pelo **Instituto Geográfico e Cartográfico de São Paulo (IGC-SP)**, que são de escala 1:50.000. Foi feito o *download* dos dados por meio do Portal DataGeo, posteriormente os dados foram descompactados e renomeados.

In [ ]:
# Carregar pacotes
import os
import urllib.request
import shutil

# Define o nome do arquivo zip a ser salvo
file = 'SP_IGC_50k'

# Define o nome do arquivo, com diretório, que será salvo
filename = os.path.join('data', file + '.zip')

# URL para baixar o arquivo zip
url = ('http://datageo.ambiente.sp.gov.br/geoserver/datageo/'+
       'LimiteMunicipal/wfs'+
       '?version=1.0.0'+
       '&request=GetFeature&outputFormat=SHAPE-ZIP'+
       '&typeName=LimiteMunicipal')

# Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
       shutil.copyfileobj(response, out_file)

# Tira da pasta zipada e renomeia arquivos
%run '../codes/files/zip_file.py'
unzip_renaming(filename, file)

# Geopandas

Pacota para manusear arquivos espaciais
http://geopandas.org/




## Lendo Arquivos

In [ ]:
import os
import geopandas as gpd

file = 'SP_IBGE_250k'
#file = 'SP_IGC_50k'

shp = os.path.join('data', file + '.shp')
dbf = os.path.join('data', file + '.dbf')

gdf = gpd.read_file(shp)
gdf

In [ ]:
%run '../codes/files/predict_encoding.py'
file_encoding = predict_encoding(dbf)
print(file_encoding)

In [ ]:
gdf.plot()

In [ ]:
gdf.crs

## Editando Colunas

In [ ]:
# Renomeia colunas
gdf=gdf.rename(index=str,
               columns={'NM_MUNICIP':'Nome_Município',
                        'CD_GEOCMU':'Geocódigo'})

gdf

In [ ]:
# Reordena colunas
gdf=gdf[['Geocódigo',
         'Nome_Município',
         'geometry']]

gdf

In [ ]:
# Define Campo com uma constante
gdf = gdf.assign(ID_Number=1)
gdf = gdf.assign(ID_Text='Text')

# Calcula o Área e Commprimento
gdf['Perímetro'] = gdf['geometry'].length
gdf['Área'] = gdf['geometry'].area

# Cálculos Simples
gdf['Cálculo'] = gdf.Perímetro * 2

# Arredonda Valores Núméricos
gdf['Perímetro Arredondado'] = round(gdf.Perímetro,0)

# Substitui textos
gdf['Nome_Município'] = gdf.Nome_Município.replace('AURIFLAMA', 'ssss')

# Pega apenas os três primeiros caracteres
gdf['Início Município'] = gdf.Nome_Município.astype(str).str[0:3]

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
gdf['ID_Município'] = [x[0:7] for x in gdf['Geocódigo']]
gdf['ID_Microregião'] = [x[0:5] for x in gdf['Geocódigo']]
gdf['ID_Mesoregião'] = [x[0:4] for x in gdf['Geocódigo']]
gdf['ID_Estado'] = [x[0:2] for x in gdf['Geocódigo']]

# Pega apenas os três primeiros caracteres de um campo de texto de maneira mais eficiente
gdf['Primeiro Caractere'] = [str(x)[0] for x in gdf['Nome_Município']]

gdf

In [ ]:
# Deleta colunas
gdf=gdf.drop(['ID_Text'], axis=1)
gdf=gdf.drop(['Perímetro Arredondado', 'ID_Number'], axis=1)

gdf

In [ ]:
gdf.bounds

## Projeções e Transformações

In [ ]:
# Projeções e Transformações
# https://epsg.io/

# get the defined CRS (Coordinate Reference Systems) of a GeoDataFrame
print(gdf.crs)
gdf.plot()

# Set the CRS (Coordinate Reference Systems) to a GeoDataFrame
#gdf.crs = from_epsg(3879)

# Project the GeoDataFrame to another CRS (Coordinate Reference Systems)
#gdf2 = gdf.to_crs({'init': 'epsg:3395'})
gdf2 = gdf.to_crs("+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")
gdf2.plot()

## Escrever Arquivos

In [ ]:
# save GeoDataFrame as a new file

gdf.to_file(os.path.join('data', 'SaoPaulo2' + '.shp'), encoding='utf-8')
gdf.to_file(os.path.join('data', 'SaoPaulo6' + '.geojson'), driver='GeoJSON', encoding='utf-8')
gdf.to_file(os.path.join('data', 'SaoPau' + '.gpkg'), layer='SaoPau', driver="GPKG")

# Exercício


## Loop
1. Selecionar 'Município' ou 'Conjunto de Municípios', por nome, código ou intersect.

2. Selecionar por localização 

3. for X in N_Escala
	Seleciona cidades que compartilham limites
	extrair cidadesadiciona campo X
	append


In [ ]:
selection = gdf[gdf['Nome_Município'] == 'SANTOS']
selection

In [ ]:
gdf['my_neighbors'] = None

for index, row in gdf.iterrows():
    #print(row)
    neighbors = gdf[gdf.geometry.touches(row['geometry'])].Nome_Município.tolist()
    #print(neighbors)
    #neighbors = neighbors.remove(row.name)
    #print(row.name)
    #print(neighbors)
    gdf.at[index, "my_neighbors"] = ", ".join(neighbors)

In [ ]:
selection = gdf[gdf['Nome_Município'] == 'SANTOS']
selection

In [ ]:
import fiona

In [ ]:
fiona.supported_drivers

In [ ]:
#https://rsandstroem.github.io/GeoMapsFoliumDemo.html
#https://michelleful.github.io/code-blog/2015/04/29/geopandas-manipulation/

# GitHub

In [ ]:
%run '~/Documents/Geodata/SourceCode/codes/git/update_github.py'

git_full('/home/michel/Documents/Geodata/SourceCode/package_geopandas', '.', 'Atualizando')